In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
sb.set()

In [2]:
np.random.seed(235623)
tmp = np.random.randn(400)
tmp[np.random.randint(400,size=79)] = None
df_1 = (pd.DataFrame(np.reshape(tmp,(100,4)),
                     columns=['col_1', 'col_2', 'col_3', 'col_4'])
        .assign(cust_id=np.random.randint(10,size=100),
                date=pd.to_datetime('2013-01-01')+pd.to_timedelta(np.random.randint(1000, size=100), unit='d')))


# Assignment 4.1
Calculate the average value of columns `col_1`, `col_2`, `col_3` and `col_4` for each `cust_id`. Insert a `NaN` if at least one of the values in a column is missing for that `cust_id`
<details><summary>Hint 1</summary>
    <p>
        A pandas groupby object has an `apply` method, that can apply a function to the entire grouping. This function takes in a DataFrame and returns a DataFrame
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        The `mean` method on a DataFrame has the parameter `skipna`, which sets whether missing values should be ignored or not
    </p>
</details>

In [5]:
df_1.drop('date', axis=1).groupby('cust_id').apply(lambda df: df.mean(skipna=False))

,col_1,col_2,col_3,col_4,cust_id
cust_id,,,,,
0,0.250502,NaN,-0.045372,NaN,0.0
1,NaN,NaN,NaN,NaN,1.0
2,-0.178076,NaN,NaN,NaN,2.0
3,NaN,-0.323611,-0.202129,NaN,3.0
4,0.081161,NaN,NaN,NaN,4.0
5,NaN,NaN,0.008253,NaN,5.0
6,NaN,NaN,NaN,NaN,6.0
7,NaN,0.393012,NaN,NaN,7.0
8,NaN,NaN,NaN,NaN,8.0


# Assignment 4.2
Create DataFrame `df_2` with columns
* `cust_id` matching the `cust_id` column in `df_1`
* `date` matching the `date` column in `df_1`
* `col_2` as the sum of `col_1` and `col_2` from `df_1`. All rows where either of the two values in the sum is missing should be replaced by 0
* `col_3` as the product of `col_3` and `col_4` from `df_1`. All rows where either of the two values in the sum is missing should be replaced by 1
<details><summary>Hint 1</summary>
    <p>
        The method `fillna` can be used to fill in missing values. It either takes a single value or a dictionary with column names as keys and fill values as values
    </p>
</details>

In [3]:
df_2 = pd.concat([df_1['cust_id'],
                  df_1['date'],
                  (df_1['col_1']+df_1['col_2']).rename('col_2'),
                  (df_1['col_3']*df_1['col_4']).rename('col_3')],
                 axis=1).fillna({'col_2':0, 'col_3':1})

# Assignment 4.3
Find the average number of missing values in a row for each `cust_id`
<details><summary>Hint 1</summary>
    <p>
        The `isnull` method return a Boolean value for each element in the DataFrame, indicating whether or not the element's value is missing
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        Summations in Python treat Boolean values as 0 for `False` and 1 for `True`
    </p>
</details>

In [18]:
df_1.drop(['cust_id', 'date'], axis=1).isnull().sum(axis=1).to_frame().assign(cust_id=df_1['cust_id']).groupby('cust_id').mean()

,0
cust_id,
0,0.555556
1,0.785714
2,0.818182
3,0.461538
4,0.500000
5,0.833333
6,0.666667
7,0.833333
8,0.500000


# Assignment 4.4
For each `cust_id` order the rows by ascending `date` and fill out missing values with the value in the previous row for that customer. If the first row for a `cust_id` has missing values, fill those with 0.
<details><summary>Hint 1</summary>
    <p>
        The `fillna` method has a method `ffill`, that fills in the previous column value instead of a missing value
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        Use `iloc` to access rows and columns by using the relative position in the DataFrame, rather than accessing values by row and column indices
    </p>
</details>

In [17]:
df_1.groupby('cust_id').apply(lambda df: pd.concat([df.sort_values('date').iloc[0:1,:].fillna(0),
                                                    df.sort_values('date').iloc[1:,:]], axis=0).fillna(method='ffill'))

col_1     col_2     col_3     col_4  cust_id       date
cust_id                                                               
0       13 -1.091561  0.415385  0.126291 -0.260567        0 2013-02-05
        89  0.698413  0.415385  0.954361 -0.858169        0 2014-01-21
        81 -0.875726 -0.433648 -1.391989 -0.230847        0 2014-02-13
        63  1.544070  1.508701  0.106314 -0.241090        0 2014-06-02
        15 -0.304988 -0.703267  0.623284 -0.241090        0 2014-07-20
        29  0.181262 -0.703267 -0.788255  1.107425        0 2014-08-28
        22 -0.312772 -0.703267 -0.349583  1.392546        0 2014-10-14
        71  1.284069 -0.118788 -0.700562  1.392546        0 2015-03-23
        50  1.131755 -1.196761  1.011789  1.537929        0 2015-06-17
1       85  0.335272  1.113743 -0.329211  0.691222        1 2013-05-21
        96 -0.380737  0.734066 -0.329211  0.691222        1 2013-07-08
        0  -0.524499  0.704507  0.677599  0.015838        1 2013-08-15
        54 -0.385718 -0.436524  0.709200 -0.290956        1 2014-01-16
        16  0.746689 -0.436524 -0.510257 -0.381972        1 2014-02-25
        59 -2.121222 -0.297502 -0.347647 -0.420233        1 2014-04-01
        87 -1.866172 -0.209883  1.345037 -1.041079        1 2014-04-13
        24 -1.866172  1.129733 -1.272725  0.928066        1 2014-05-31
        49 -0.658703  1.129733 -1.272725  0.613986        1 2014-09-24
        95 -0.658703 -1.394385 -1.272725 -0.712081        1 2014-12-20
        8   0.499630 -1.394385 -1.272725 -0.545488        1 2015-02-17
        18  0.036848 -0.265109  0.060617  0.903944        1 2015-07-01
        57 -0.261775  0.200070 -0.580427 -0.857140        1 2015-08-15
        53 -2.447200 -0.796510 -0.580427 -0.152506        1 2015-09-22
2       84  1.009788 -1.215657  1.251394 -0.014864        2 2013-10-04
        60 -1.529843  0.308198  1.251394 -0.014864        2 2013-11-23
        47 -0.618017  0.308198  0.468105 -0.014864        2 2014-01-12
        92 -0.626156  2.183808 -1.018444  0.326317        2 2014-01-25
        3   0.035907  2.183808 -0.222607 -0.548972        2 2014-02-09
        2  -0.989378  2.183808  0.884415  1.991800        2 2014-02-13
        44  0.425030  0.364154  0.489320  1.991800        2 2014-09-29
...              ...       ...       ...       ...      ...        ...
6       38 -0.397530  0.656605 -0.375104 -0.720870        6 2013-05-05
        75  0.513152  0.277630 -1.756759 -0.720870        6 2013-09-11
        98  0.513152  0.277630 -1.756759 -1.759565        6 2014-05-18
        43 -1.464016  0.277630 -2.869922 -1.087650        6 2014-08-23
        25 -0.969174  0.832344  1.244520 -0.395275        6 2014-08-30
        5   0.818104  1.784504 -0.204159  0.292151        6 2014-09-09
        26  0.469555 -0.951555  1.707883  1.170039        6 2015-01-19
        1   0.257024  1.497734  0.721475 -0.255969        6 2015-07-05
7       30 -0.259132  0.129174  0.000000  0.413938        7 2013-03-31
        14 -0.259132  0.209356 -0.268664  0.413938        7 2013-04-25
        52  0.976334  1.208011 -0.104840  0.413938        7 2013-06-04
        12  1.143012  0.161017 -1.736760  0.104054        7 2015-07-27
        67 -0.321362  0.615908 -0.125008  0.217132        7 2015-08-26
        40  0.278877  0.034604  1.113949  0.217132        7 2015-09-07
8       93  1.753250  1.664299  0.000000  1.794920        8 2013-03-11
        69  1.667675 -1.693651  0.600177  0.443752        8 2013-03-13
        74 -0.301393  0.036381  0.341445 -0.314656        8 2013-08-22
        91 -0.214450 -1.109838  0.123646 -0.314656        8 2013-12-20
        94 -1.497976 -0.474142 -1.222543  0.494378        8 2014-10-21
        65  0.803750 -0.474142 -1.607430 -0.561365        8 2014-11-19
        11  0.803750 -1.051291 -0.177579  0.711529        8 2015-06-05
        55  0.280382  0.008674  0.414681 -0.581990        8 2015-07-29
9       51 -0.990559 -0.787799  0.000000 -0.271210        9 2013-08-14
        23 -0.990559 -2.101531  0.00000

# Assignment 4.5
Reorder the columns in `df_2` to match those in `df_1`, inserting columns of missing values for all column names not already in `df_2`.
<details><summary>Hint 1</summary>
    <p>
        The `reindex` method reorders columns and inserts missing values
    </p>
</details>

In [6]:
df_2 = df_2.reindex(df_1.columns, axis=1)